# Formateo de Datos (Tidy Data)
Como mencionamos anteriormente. Nos aprovecharemos de la forma de la base de datos para darle un formato más adecuado para su análisis. La estructura constará de usar un múlti indice en la parte de las columnas y de renglonres los municipios. 

## Extracción de Datos crudos (Raw Data)

In [1]:
import pandas as pd
import numpy as np
from transform import transform as tr

tr.transform_mun()

ModuleNotFoundError: No module named 'transform'